In [96]:
# Import packages
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import networkx as nx
import pandas as pd
import numpy as np
import scipy as sp
import datetime as dt
import community
from shapely.geometry import Polygon
import importlib

import DDOM # Data Driven Occupational Mobility
import model_cal
importlib.reload(DDOM)
import random
import math

import simpy

import cmocean as cmo


%matplotlib inline

In [97]:
# Read the data from data_processing.ipynb
sa_calibration_data = pd.read_csv('../Data_Labour/calibration_data.csv')
employment_SSYK = pd.read_csv('../Data_Labour/occupational_employment.csv', sep = ',')
SSYK_shock = pd.read_csv('../Data_Labour/occupation_shock.csv', sep = ',')

G = nx.read_graphml('../Data_Labour/Occ_mob_sweden.graphml')

In [98]:
def deterministic_simulation(individual, G, period, timestep, empirical_data, k, L, avg_hours_0, shock_start, attributes, calibration_output = False):
    #set_attributes(G, data)
    # This needs to be put into the network (used as starting point)

    for key, value in attributes.items():
        nx.set_node_attributes(G, value, str(key))

    timesteps = round(period*52/timestep)*3 # Total amount of steps
    T_steps = round(period*52/timestep) # Steps during simulated business cycle
    



    vacancies = nx.get_node_attributes(G, 'vacancies')
    employed = nx.get_node_attributes(G, 'employed')

    demand_0 = {}

    for key in vacancies.keys():
        demand_0[key] = vacancies[key] + employed[key] 

    vac_data = []
    emp_data = []
    unemp_data = []
    td_data = []

    # Variables to calculate the post shock demand
    risk_factor = nx.get_node_attributes(G, 'comp_prob')
    average_hours_worked_0 = avg_hours_0
    

    final_hours_worked = {}

    for occupation in risk_factor.keys():
        final_hours_worked[occupation] = average_hours_worked_0*employed[occupation]*(1-risk_factor[occupation])

    final_average_hours_worked = sum(final_hours_worked.values())/L

    # Post shock demand
    final_demand = {occupation:hours/final_average_hours_worked for occupation, hours in final_hours_worked.items()}

    occupations = list(G.nodes())
    time = dt.datetime.now()

    a = individual[0]
    delta_u = individual[1]
    delta_ny = individual[2]
    gamma_u = individual[3]

    print('Simulation started at: ', time)
    for t in range(timesteps):
        ny = nx.get_node_attributes(G, 'vacancies')
        u = nx.get_node_attributes(G, 'unemployed')
        e = nx.get_node_attributes(G, 'employed')
        A = nx.get_edge_attributes(G, 'weight')

        s = {}
        f = {}
        for j in occupations:
            s[j] = []
            for i in G.predecessors(j):
                ny_A_sum = sum([ny[k]*A[(i,k)] for k in G.neighbors(i)])
                if ny_A_sum == 0:
                    s[j].append(0)
                else:
                    s[j].append(u[i]*ny[j]*A[(i,j)]/ny_A_sum)

            s[j] = sum(s[j])
            for i in G.predecessors(j):
                ny_A_sum = sum([ny[k]*A[(i,k)] for k in G.neighbors(i)])
                if s[j]*ny_A_sum == 0:
                    f[(i,j)] = 0
                else:
                    f[(i,j)] = u[i]*(ny[j]**(2))*A[(i,j)]*(1 - math.exp(-s[j]/ny[j]))/(s[j]*ny_A_sum)

        new_e = {}
        new_u = {}
        new_ny = {}
        
        target_demand = nx.get_node_attributes(G, 'target_demand')
        current_demand = {}

        for i in occupations:
            current_demand[i] = ny[i] + e[i]
            demand_diff = max(0, current_demand[i] - target_demand[i])

            f_i = sum([f[(j,i)] for j in G.predecessors(i)])

            new_e[i] = e[i] - delta_u*e[i] + (1 - delta_u)*gamma_u*demand_diff + f_i

            f_j = sum([f[(i,j)] for j in G.successors(i)])

            new_u[i] = u[i] + delta_u*e[i] + (1 - delta_u)*gamma_u*demand_diff - f_j

            demand_diff = max(0, target_demand[i]-current_demand[i])

            new_ny[i] = ny[i] + delta_ny*e[i] + (1-delta_ny)*gamma_u*demand_diff - f_i

        nx.set_node_attributes(G, new_ny, 'vacancies')
        nx.set_node_attributes(G, new_e, 'employed')
        nx.set_node_attributes(G, new_u, 'unemployed')

        vac_data.append(nx.get_node_attributes(G, 'vacancies'))
        unemp_data.append(nx.get_node_attributes(G, 'unemployed'))
        emp_data.append(nx.get_node_attributes(G, 'employed'))
        td_data.append(target_demand)

        if T_steps < t:
            DDOM.update_target_demand(G, demand_0, t, T_steps, a)
        # order should be checked and changed
        # if t > shock_start:
        #    shock(G, demand_0, final_demand, t, t_0, k)

    model_data = {'vacancies': vac_data, 'unemployment': unemp_data, 'employment':emp_data}
    # Empirical data
    e_vac_rate = empirical_data['sa_vac_rate']
    e_unemployed = empirical_data['u_trend']
    e_seq = [(u, e_vac_rate.iloc[i]) for i, u in enumerate(e_unemployed)]#
    e_u_max = max(e_unemployed)
    e_u_min = min(e_unemployed)
    e_vac_max = max(e_vac_rate)
    e_vac_min = min(e_vac_rate)

    

    A_e = Polygon(e_seq).buffer(0)

    # Cost is a vector of deviations from goal. Should be 0
    cost = DDOM.calibration_calculation(empirical_data, model_data, A_e, T_steps)

    # m_td = [sum(demand.values()) for demand in td_data]
    # plt.plot(m_td)
    # plt.show()


    cost['A_e'] = A_e.area
    # if cost['intersection'] != 'N/A' and cost['union'] != 'N/A' and cost['A_m'] != 'N/A':
    #     fitness = [
    #         abs(cost['A_e']-cost['A_m']),
    #         abs(e_u_max - cost['m_u_max']),
    #         abs(e_u_min - cost['m_u_min']),
    #         abs(e_vac_max - cost['m_vac_max']),
    #         abs(e_vac_min - cost['m_vac_min']),
    #         abs(cost['A_e'] - cost['intersection']),
    #         abs(cost['A_e'] - cost['union']),
    #         cost['union'] - cost['intersection']
    #     ]
    # elif cost['A_m'] != 'N/A':
    #     fitness = [
    #         abs(cost['A_e']-cost['A_m']),
    #         abs(e_u_max - cost['m_u_max']),
    #         abs(e_u_min - cost['m_u_min']),
    #         abs(e_vac_max - cost['m_vac_max']),
    #         abs(e_vac_min - cost['m_vac_min'])
    #     ]
    # else:
    #     fitness = [
    #         abs(e_u_max - cost['m_u_max']),
    #         abs(e_u_min - cost['m_u_min']),
    #         abs(e_vac_max - cost['m_vac_max']),
    #         abs(e_vac_min - cost['m_vac_min'])
    #     ]
    fitness = [
    abs(e_u_max - cost['m_u_max']),
    abs(e_u_min - cost['m_u_min']),
    abs(e_vac_max - cost['m_vac_max']),
    abs(e_vac_min - cost['m_vac_min'])
    ]
    fitness = np.linalg.norm(fitness)

    vac_data = pd.DataFrame(vac_data)
    unemp_data = pd.DataFrame(unemp_data)
    emp_data = pd.DataFrame(emp_data)
    time = dt.datetime.now()- time
    print('Simulation took: ', time)
    cost['time'] = time
    if calibration_output == 'evo':
        return fitness
    elif calibration_output == 'True':
        return cost
    else:
        return {'vacancy_data': vac_data, 'unemployment_data': unemp_data, 'employment_data': emp_data, 'cost': cost}

In [99]:
def eval_func(individual):
    while individual[1] < individual[2]:
        individual[1] += 0.01
    employment = employment_SSYK[['SSYK', '2014']]
    employment = {str(employment['SSYK'].iloc[i]):employment['2014'].iloc[i] for i in range(len(employment))}
    node_names = list(G.nodes())

    # setup network
    employed = {str(name):e for name,e in employment.items() if str(name) in node_names}
    unemployed = {name:0 for name in node_names}
    vacancies = {name:0 for name in node_names}
    applications = {name:[] for name in node_names}
    target_demand = {str(name):e for name,e in employment.items() if str(name) in node_names}
    of_data = SSYK_shock.groupby(by = ['ssyk3'], axis = 0).mean()
    of_data = of_data.to_dict()['Computerisation Probability']
    attributes = {'employed':employed, 'unemployed':unemployed, 'vacancies':vacancies, 'applications':applications,
                    'target_demand':target_demand, 'comp_prob':of_data}

    # Save parameters


    # Calibration data
    start = 18
    end = 59
    empirical_data = sa_calibration_data.iloc[start:end]
    parameters = {}
    
    parameters['T'] = 15
    # Shock parameters (not there yet)
    parameters['t_0'] = 10
    parameters['years'] = parameters['T'] + parameters['t_0']
    parameters['k'] = 1
    parameters['L'] = 1
    parameters['avg_hours_0'] = 1
    parameters['shock_start'] = 1
    parameters['timestep'] = 6

    calibration_output = 'evo'

    # Run simulation
    fitness = deterministic_simulation(individual, G, parameters['T'], parameters['timestep'], empirical_data,
                    parameters['k'], parameters['L'], parameters['avg_hours_0'], parameters['shock_start'],
                    attributes, calibration_output)
    print('a:', individual[0], 'delta_u:', individual[1], 'delta_ny:', individual[2], 'gamma_u:', individual[3], 'fitness:', fitness)

    string = 'a: ' + str(individual[0]) + ' delta_u: ' + str(individual[1]) + ' delta_ny: ', str(individual[2]) + ' gamma_u: ' + str(individual[3]) + ' fitness: ' + str(fitness)
    
    f = open("../Data_Labour/parameters_fitness.txt","a")
    f.write(string + "\n")
    results.append({'fitness':fitness, 'individual':individual})

    return fitness,


In [101]:

#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

import array
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, typecode='b', fitness=creator.FitnessMin)

toolbox = base.Toolbox()

open("../Data_Labour/parameters_fitness.txt","w+")

# parameters['a'] = [0.01, 0.03]
# parameters['delta_u'] = [0.0065, 0.01]
# parameters['delta_ny'] = [0.0055, 0.006, 0.0065]
# parameters['gamma_u'] = [0.15, 0.16, 0.165]
# parameters['timestep'] = [3.3]
# parameters['gamma_ny'] = parameters['gamma_u']

# Attribute generator
# toolbox.register("attr", random.uniform, 0, 0.2)



# func_seq = [lambda:random.uniform(0.01, 0.2), lambda:random.uniform(0.005, 0.09), lambda:random.uniform(0.001, 0.05), lambda:random.uniform(0.05, 0.2)]

# a: 0.022252957991699827 delta_u: 0.01075412432464562 delta_ny: 0.009035380457934611 gamma_u: 0.13015569282507672
# a: 0.01927319683779494 delta_u: 0.017759426492261806 delta_ny: 0.009603112274559051 gamma_u: 0.1436019268182242
# a: 0.019498541354135167 delta_u: 0.017156297087313012 delta_ny: 0.009231119344329925 gamma_u: 0.13942978729410904

# def nvariate_seq(mu, sigma):
#     abs(random.normalvariate(mu, sigma))

func_seq = [lambda : abs(random.normalvariate(0.04, 0.02)), lambda : abs(random.normalvariate(0.018, 0.001)), lambda : abs(random.normalvariate(0.005, 0.001)), lambda : abs(random.normalvariate(0.16, 0.01))]

# func_seq = [nvariate_seq(0.04, 0.02)]

# a: 0.05 delta_u: 0.01 delta_ny: 0.005 gamma_u: 0.13
# a: 0.02848946154491157 delta_u: 0.010664600228730777 delta_ny: 0.0023964714590789093 gamma_u: 0.1282955755725513

# Structure initializers
toolbox.register("individual", tools.initCycle, creator.Individual, func_seq, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


toolbox.register("evaluate", eval_func)
toolbox.register("mate", tools.cxUniform, indpb = 0.5)
toolbox.register("mutate", tools.mutGaussian, mu = 0.01, sigma = 0.01, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

import multiprocessing


def main():
    random.seed(64)

    results = []

    pool = multiprocessing.Pool()
    toolbox.register("map", pool.map)

    
    pop = toolbox.population(n=40)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    pop, log = algorithms.eaMuPlusLambda(pop, toolbox, mu = 10, lambda_ = 20, cxpb=0.5, mutpb=0.4, ngen=15, 
                                   stats=stats, halloffame=hof, verbose=True)
    
    return pop, log, hof

if __name__ == "__main__":
    pop, lpg, hof = main()


KeyboardInterrupt: 

In [ ]:
lpg

In [ ]:
results